<a href="https://colab.research.google.com/github/kgautam2103/Bulk_Email_tool_public/blob/main/BulkEmailTool_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# Import Python Packages
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd
import time
import base64
from datetime import datetime
#from google.colab import files

# Set Global Variables
current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_email_output'
gmail_user = ''
gmail_password_encoded = ''
is_attachment=input('Attachment needed ? Enter true or false: ').lower()
if is_attachment == 'true':
  attached_file_name_with_path = input('enter the path along with attachment name like /content/MeditationGuide.pdf : ')
  last_index_slash = max(index for index, item in enumerate(attached_file_name_with_path) if item == '/')
  attached_file_name = attached_file_name_with_path[last_index_slash+1:]

counter = 1

base64_bytes = gmail_password_encoded.encode("ascii")  
pswd_string_bytes = base64.b64decode(base64_bytes)
gmail_password = pswd_string_bytes.decode("ascii")

#server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(gmail_user, gmail_password)
input_ws=input('please enter inputdata or sampledata: ')
#input_ws = 'sampledata'

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1E_chz7BaaiBDjDSn_x2IKt1UocXJbUVOqc_Fx52uoTY/edit#gid=0')

input_sheet = wb.worksheet(input_ws)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['email','name','subject','message','regards','greeting'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['email_id','name', 'email_sent','error'],1)

#attach_file_name = '/content/MeditationGuide.pdf'
#attach_file = open(attach_file_name, 'rb') # Open the file as binary mode
#attach_file=files.upload()


for index, row in inputdata_df.iterrows():
  counter = counter+1
  from_email = gmail_user
  to_email = str(row['email'])
  email_subject = str(row['subject'])
  email_name = str(row['name'])
  email_greet = str(row['greeting'])+" "+email_name+","
  email_message_body = str(row['message'])
  email_regards = str(row['regards'])
  email_success = ''
  email_error = ''

  email_message = '''
  %s

  %s

  Regards,
  %s
  ''' % (email_greet, email_message_body, email_regards) 

  #print(email_message)

  msg = MIMEMultipart()
  # setup the parameters of the message
  msg['From']=from_email
  msg['To']=to_email
  msg['Subject']=email_subject

  # add in the message body
  msg.attach(MIMEText(email_message, 'plain'))

  #add attachment
  if is_attachment == 'true':
    #attach_file_name = '/content/MeditationGuide.pdf'
    attach_file = open(attached_file_name_with_path, 'rb') # Open the file as binary mode
    payload = MIMEBase('application', 'octate-stream')
    payload.set_payload((attach_file).read())
    encoders.encode_base64(payload) #encode the attachment
    #add payload header with filename
    payload.add_header('content-disposition', 'attachment', filename=attached_file_name)
    msg.attach(payload)

  try:
    # Sent Email  
    server.send_message(msg)
    time.sleep(3)
  except Exception as e:
    email_success = 'False'
    email_error = str(e)
    print(e)
  else:
    email_success = 'True'
  finally:
    cell_value_list = [to_email,email_name,email_success,email_error]
    output_sheet.insert_row(cell_value_list,counter)

  del msg

server.close()

Attachment needed ? Enter true or false: false
please enter inputdata or sampledata: inputdata


In [ ]:
import base64
  
sample_string = ""
sample_string_bytes = sample_string.encode("ascii")
  
base64_bytes = base64.b64encode(sample_string_bytes)
base64_string = base64_bytes.decode("ascii")

base64_string

'bmlybWFsQDIxMDM='